In [1]:
import numpy as np
import pandas as pd

http://ethen8181.github.io/machine-learning/association_rule/apriori.html

# Load SPFM data and conver it to Panda dataframe

In [2]:
%%sh
wget http://www.philippe-fournier-viger.com/spmf/datasets/SIGN.txt

--2020-10-18 01:39:53--  http://www.philippe-fournier-viger.com/spmf/datasets/SIGN.txt
Resolving www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)... 74.208.236.167
Connecting to www.philippe-fournier-viger.com (www.philippe-fournier-viger.com)|74.208.236.167|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236727 (231K) [text/plain]
Saving to: ‘SIGN.txt’

     0K .......... .......... .......... .......... .......... 21%  607K 0s
    50K .......... .......... .......... .......... .......... 43% 1.18M 0s
   100K .......... .......... .......... .......... .......... 64% 2.18M 0s
   150K .......... .......... .......... .......... .......... 86% 2.59M 0s
   200K .......... .......... .......... .                    100% 3.56M=0.2s

2020-10-18 01:39:54 (1.30 MB/s) - ‘SIGN.txt’ saved [236727/236727]



In [3]:
spfm_data='SIGN.txt'

In [4]:
#Read SPFM raw data
f = open(spfm_data, 'r+')
raw_lines = [line.strip() for line in f.readlines()]
f.close()

In [126]:
#Seperator is -1 and -2 is end of line -->convert into CSV 
itemset_dict={}
i=0
max_cols=0
for line in raw_lines:
  itemset=[int(item.strip()) for item in line.split('-1')[:-1] ]
  itemset_dict[i]=itemset
  if max_cols < len(itemset):
    max_cols=len(itemset)
  i=i+1

In [6]:
#Make each row of same number of columns by padding zeros upto max_columns
from itertools import repeat
for key in itemset_dict:
  itemset_dict[key].extend(repeat(0, max_cols - len(itemset_dict[key])))

In [7]:
len(itemset_dict[1])

94

In [8]:
itemset_list=[]
for key in itemset_dict:
  itemset_list.append(itemset_dict[key])

In [9]:
df=pd.DataFrame(np.array(itemset_list)).head()

**Get unique items from all transactions**

In [127]:
from itertools import chain 

item_matrix=np.array(itemset_list)
unique_items=list(set(chain(*(item_matrix))))

In [128]:
#Number of items matching with Item count provided at http://www.philippe-fournier-viger.com/spmf/index.php?link=datasets.php
unique_items.remove(0)
len(unique_items)

267

**Bit Array Matrix**

In [429]:
from sklearn.feature_extraction.text import CountVectorizer
corpus =['I1,I2,I5',
'I2,I4',
'I2,I3',
'I1,I2,I4',
'I1,I3',
'I2,I3',
'I1,I3',
'I1,I2,I3,I5',
'I1,I2,I3']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
index_col=['T'+str(i+1)  for i in range(len(corpus))]
print(index_col)
print(X.toarray())

['i1', 'i2', 'i3', 'i4', 'i5']
['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9']
[[1 1 0 0 1]
 [0 1 0 1 0]
 [0 1 1 0 0]
 [1 1 0 1 0]
 [1 0 1 0 0]
 [0 1 1 0 0]
 [1 0 1 0 0]
 [1 1 1 0 1]
 [1 1 1 0 0]]


In [430]:
k=1
C={}
L={}

In [431]:
C[k] = pd.DataFrame(data=X.toarray(), index=index_col ,columns=vectorizer.get_feature_names())
C[k]

,i1,i2,i3,i4,i5
T1,1,1,0,0,1
T2,0,1,0,1,0
T3,0,1,1,0,0
T4,1,1,0,1,0
T5,1,0,1,0,0
T6,0,1,1,0,0
T7,1,0,1,0,0
T8,1,1,1,0,1
T9,1,1,1,0,0


In [432]:
min_sup=2
L[k]=C[k].loc[:, (C[k] .sum(axis=0) >= min_sup)]  #Sum of columns is greater then minimum support
L[k]=L[k][L[k].sum(axis=1) >= min_sup]            #Sum of rows is greater then minimum support
L[k]

,i1,i2,i3,i4,i5
T1,1,1,0,0,1
T2,0,1,0,1,0
T3,0,1,1,0,0
T4,1,1,0,1,0
T5,1,0,1,0,0
T6,0,1,1,0,0
T7,1,0,1,0,0
T8,1,1,1,0,1
T9,1,1,1,0,0


In [433]:
print(C[1] )

print(L[1])

    i1  i2  i3  i4  i5
T1   1   1   0   0   1
T2   0   1   0   1   0
T3   0   1   1   0   0
T4   1   1   0   1   0
T5   1   0   1   0   0
T6   0   1   1   0   0
T7   1   0   1   0   0
T8   1   1   1   0   1
T9   1   1   1   0   0
    i1  i2  i3  i4  i5
T1   1   1   0   0   1
T2   0   1   0   1   0
T3   0   1   1   0   0
T4   1   1   0   1   0
T5   1   0   1   0   0
T6   0   1   1   0   0
T7   1   0   1   0   0
T8   1   1   1   0   1
T9   1   1   1   0   0


In [434]:
freq_items_set={}
freq_items_set[k]=L[k].columns.tolist()
freq_items_set[k]

['i1', 'i2', 'i3', 'i4', 'i5']

In [411]:
# Function which returns subset or r length from n 
from itertools import combinations 
def rSubset(arr, r): 
  return frozenset(list(combinations(arr, r)))

In [435]:
itemset_col_dict=rSubset(freq_items_set[k], k+1)
itemset_col_dict

frozenset({('i1', 'i2'),
           ('i1', 'i3'),
           ('i1', 'i4'),
           ('i1', 'i5'),
           ('i2', 'i3'),
           ('i2', 'i4'),
           ('i2', 'i5'),
           ('i3', 'i4'),
           ('i3', 'i5'),
           ('i4', 'i5')})

In [436]:
k=k+1
k

2

In [437]:
C[k]=L[k-1].copy()

In [438]:
# list of columns
columns = C[k].columns

# Creating a new pd.DataFrame
new_df = C[k].copy()

for itemset in itemset_col_dict:
  combination_string = ",".join(itemset)
  new_df[combination_string] = C[k][itemset[1]] & C[k][itemset[0]]
  #print(combination_string)

new_df.drop(columns,axis=1,inplace=True)

In [439]:
C[k]=new_df.copy()

In [440]:
C[k]

,"i1,i3","i1,i4","i4,i5","i3,i4","i2,i4","i1,i5","i3,i5","i2,i3","i2,i5","i1,i2"
T1,0,0,0,0,0,1,0,0,1,1
T2,0,0,0,0,1,0,0,0,0,0
T3,0,0,0,0,0,0,0,1,0,0
T4,0,1,0,0,1,0,0,0,0,1
T5,1,0,0,0,0,0,0,0,0,0
T6,0,0,0,0,0,0,0,1,0,0
T7,1,0,0,0,0,0,0,0,0,0
T8,1,0,0,0,0,1,1,1,1,1
T9,1,0,0,0,0,0,0,1,0,1


In [441]:
s=C[k].reset_index().melt('index')
pd.crosstab(index=s['index'],columns=s.variable,values=s.value,aggfunc='sum',margins=True)

variable,"i1,i2","i1,i3","i1,i4","i1,i5","i2,i3","i2,i4","i2,i5","i3,i4","i3,i5","i4,i5",All
index,,,,,,,,,,,
T1,1,0,0,1,0,0,1,0,0,0,3
T2,0,0,0,0,0,1,0,0,0,0,1
T3,0,0,0,0,1,0,0,0,0,0,1
T4,1,0,1,0,0,1,0,0,0,0,3
T5,0,1,0,0,0,0,0,0,0,0,1
T6,0,0,0,0,1,0,0,0,0,0,1
T7,0,1,0,0,0,0,0,0,0,0,1
T8,1,1,0,1,1,0,1,0,1,0,6
T9,1,1,0,0,1,0,0,0,0,0,3


In [442]:
L[k]=C[k].loc[:, (C[k] .sum(axis=0) >= min_sup)]  #Sum of columns is greater then minimum support
L[k]=L[k][L[k].sum(axis=1) >= min_sup]            #Sum of rows is greater then minimum support
L[k]

,"i1,i3","i2,i4","i1,i5","i2,i3","i2,i5","i1,i2"
T1,0,0,1,0,1,1
T4,0,1,0,0,0,1
T8,1,0,1,1,1,1
T9,1,0,0,1,0,1


In [443]:
print(C[2] )

print(L[2])

    i1,i3  i1,i4  i4,i5  i3,i4  i2,i4  i1,i5  i3,i5  i2,i3  i2,i5  i1,i2
T1      0      0      0      0      0      1      0      0      1      1
T2      0      0      0      0      1      0      0      0      0      0
T3      0      0      0      0      0      0      0      1      0      0
T4      0      1      0      0      1      0      0      0      0      1
T5      1      0      0      0      0      0      0      0      0      0
T6      0      0      0      0      0      0      0      1      0      0
T7      1      0      0      0      0      0      0      0      0      0
T8      1      0      0      0      0      1      1      1      1      1
T9      1      0      0      0      0      0      0      1      0      1
    i1,i3  i2,i4  i1,i5  i2,i3  i2,i5  i1,i2
T1      0      0      1      0      1      1
T4      0      1      0      0      0      1
T8      1      0      1      1      1      1
T9      1      0      0      1      0      1


In [444]:
L[k].columns.tolist()

['i1,i3', 'i2,i4', 'i1,i5', 'i2,i3', 'i2,i5', 'i1,i2']

In [445]:
temp_vectortizer = vectorizer.fit_transform(L[k].columns.tolist())

In [446]:
freq_items_set[k]=vectorizer.get_feature_names()
itemset_col_dict=rSubset(freq_items_set[k],k+1)
itemset_col_dict

frozenset({('i1', 'i2', 'i3'),
           ('i1', 'i2', 'i4'),
           ('i1', 'i2', 'i5'),
           ('i1', 'i3', 'i4'),
           ('i1', 'i3', 'i5'),
           ('i1', 'i4', 'i5'),
           ('i2', 'i3', 'i4'),
           ('i2', 'i3', 'i5'),
           ('i2', 'i4', 'i5'),
           ('i3', 'i4', 'i5')})

In [447]:
k=k+1
k

3

In [448]:
C[k]=L[k-1].copy()

In [449]:
C[k]

,"i1,i3","i2,i4","i1,i5","i2,i3","i2,i5","i1,i2"
T1,0,0,1,0,1,1
T4,0,1,0,0,0,1
T8,1,0,1,1,1,1
T9,1,0,0,1,0,1


In [458]:
# Creating a new pd.DataFrame
new_df = C[k].copy()
new_df.index

Index(['T1', 'T4', 'T8', 'T9'], dtype='object')

In [488]:
C1_Valid_rows=C[1][C[1].index.isin(new_df.index)]
C1_Valid_rows[freq_items_set[1]]

,i1,i2,i3,i4,i5
T1,1,1,0,0,1
T4,1,1,0,1,0
T8,1,1,1,0,1
T9,1,1,1,0,0


In [489]:
freq_items_set[1]

['i1', 'i2', 'i3', 'i4', 'i5']

In [490]:
C1_Valid_rows

,i1,i2,i3,i4,i5
T1,1,1,0,0,1
T4,1,1,0,1,0
T8,1,1,1,0,1
T9,1,1,1,0,0


In [491]:
# Creating a new pd.DataFrame
new_df = C1_Valid_rows.copy()
for itemset in itemset_col_dict:
  combination_string = ",".join(itemset)
  print(combination_string)
  new_df[combination_string] = new_df[itemset[0]] & new_df[itemset[1]] & new_df[itemset[2]]

new_df.drop(columns,axis=1,inplace=True)
new_df

i1,i3,i5
i2,i3,i5
i1,i3,i4
i1,i2,i4
i2,i4,i5
i1,i2,i3
i2,i3,i4
i3,i4,i5
i1,i2,i5
i1,i4,i5


,"i1,i3,i5","i2,i3,i5","i1,i3,i4","i1,i2,i4","i2,i4,i5","i1,i2,i3","i2,i3,i4","i3,i4,i5","i1,i2,i5","i1,i4,i5"
T1,0,0,0,0,0,0,0,0,1,0
T4,0,0,0,1,0,0,0,0,0,0
T8,1,1,0,0,0,1,0,0,1,0
T9,0,0,0,0,0,1,0,0,0,0


In [492]:
C[k]=new_df.copy()
C[k]

,"i1,i3,i5","i2,i3,i5","i1,i3,i4","i1,i2,i4","i2,i4,i5","i1,i2,i3","i2,i3,i4","i3,i4,i5","i1,i2,i5","i1,i4,i5"
T1,0,0,0,0,0,0,0,0,1,0
T4,0,0,0,1,0,0,0,0,0,0
T8,1,1,0,0,0,1,0,0,1,0
T9,0,0,0,0,0,1,0,0,0,0


In [493]:
s=C[k].reset_index().melt('index')
pd.crosstab(index=s['index'],columns=s.variable,values=s.value,aggfunc='sum',margins=True)

variable,"i1,i2,i3","i1,i2,i4","i1,i2,i5","i1,i3,i4","i1,i3,i5","i1,i4,i5","i2,i3,i4","i2,i3,i5","i2,i4,i5","i3,i4,i5",All
index,,,,,,,,,,,
T1,0,0,1,0,0,0,0,0,0,0,1
T4,0,1,0,0,0,0,0,0,0,0,1
T8,1,0,1,0,1,0,0,1,0,0,4
T9,1,0,0,0,0,0,0,0,0,0,1
All,2,1,2,0,1,0,0,1,0,0,7


In [494]:
L[k]=C[k].loc[:, (C[k] .sum(axis=0) >= min_sup)]  #Sum of columns is greater then minimum support
L[k]=L[k][L[k].sum(axis=1) >= min_sup]            #Sum of rows is greater then minimum support
L[k]

,"i1,i2,i3","i1,i2,i5"
T8,1,1


**Rule Generation**

In [506]:
from itertools import chain, combinations

def getNonEmptySubsets(iterable):    
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

L[k].columns.tolist()    

['i1,i2,i3', 'i1,i2,i5']

In [507]:
for item in L[k].columns.tolist():
  tup=tuple(item.split(','))
  lst=list(getNonEmptySubsets(tup))
  print(lst)

[('i1',), ('i2',), ('i3',), ('i1', 'i2'), ('i1', 'i3'), ('i2', 'i3'), ('i1', 'i2', 'i3')]
[('i1',), ('i2',), ('i5',), ('i1', 'i2'), ('i1', 'i5'), ('i2', 'i5'), ('i1', 'i2', 'i5')]


In [509]:
min_confidence=60/100
min_confidence

0.6